# **TASK-3: RECOMMENDATION SYSTEM

---



In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Example initial dataset: Users' ratings for various movies
data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 4, 4],
    'movie_id': [1, 2, 3, 1, 2, 4, 2, 3, 1, 4],
    'rating': [5, 4, 3, 4, 5, 2, 2, 3, 5, 4]
}

# Convert to a DataFrame
df = pd.DataFrame(data)

# Create a pivot table with users as rows and movies as columns
user_movie_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')

# Fill NaN values with 0 (could also use other methods like mean rating)
user_movie_matrix.fillna(0, inplace=True)

# Compute the cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)

# Convert the similarity matrix to a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def update_user_ratings(user_id, movie_ratings):
    """
    Update the user-movie matrix with real-time user ratings.

    :param user_id: ID of the user
    :param movie_ratings: Dictionary of {movie_id: rating} for the user
    """
    global user_movie_matrix

    for movie_id, rating in movie_ratings.items():
        user_movie_matrix.loc[user_id, movie_id] = rating

    # Fill NaN values with 0
    user_movie_matrix.fillna(0, inplace=True)

    # Recompute the cosine similarity matrix
    global user_similarity_df
    user_similarity = cosine_similarity(user_movie_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def get_recommendations(user_id, num_recommendations=2):
    """
    Get movie recommendations for a given user.

    :param user_id: ID of the user
    :param num_recommendations: Number of recommendations to return
    :return: Recommended movies as a pandas Series
    """
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)


    similar_users = similar_users.drop(user_id)


    similar_user_ids = similar_users.index
    similar_users_movies = user_movie_matrix.loc[similar_user_ids]


    movie_recommendations = similar_users_movies.sum().sort_values(ascending=False)


    user_rated_movies = user_movie_matrix.loc[user_id]
    movie_recommendations = movie_recommendations.drop(user_rated_movies[user_rated_movies > 0].index, errors='ignore')


    return movie_recommendations.head(num_recommendations)


user_id = 5
new_ratings = {
    1: 5,
    2: 3
}


update_user_ratings(user_id, new_ratings)

recommended_movies = get_recommendations(user_id=user_id, num_recommendations=2)
print("Recommended Movies for User 5:", recommended_movies)

Recommended Movies for User 5: movie_id
3    6.0
4    6.0
dtype: float64
